<a href="https://colab.research.google.com/github/adalbertii/Modele-klasyfikacyjne/blob/main/keras_classification_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Klasyfikacja binarna**

Oparta na modelu neuronowym

Rozpoznawanie cukrzycy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Pobranie danych i ich uproszczona analiza**

In [ ]:
# zamontować dysk Google
df = pd.read_csv('/content/drive/MyDrive/dane/diabetes.csv')
print(df.shape)

In [ ]:

df.head()

In [ ]:
# analiza statystyczna zbioru danych
df.describe()

In [ ]:
# sprawdzamy korelacje zmiennych
df.corr()['Outcome'].sort_values(ascending=False)

In [ ]:
#Sprawdzenie korelacji

print('\033[1mMatryca korelacji zmiennych'.center(100))
plt.figure(figsize=[8,6])
sns.heatmap(df.corr(), annot=True, vmin=-1, vmax=1, center=0) #cmap='BuGn'
plt.show()

In [ ]:
#sprawdzenie rozkładu unikalnych wartości zmiennej objaśnianej
print(np.unique(df['Outcome'], return_counts=True))

In [ ]:
#weryfikacja zależności pomiedzy zmiennymi
#'Outcome', 'BMI', 'Age','Pregnancies'
g = sns.pairplot(df[[ 'Outcome','Glucose','BMI']])
#plt.title('Wykresy par dla wszystkich zmiennych')
g.map_upper(sns.kdeplot, levels=4, color=".2")
plt.show()

Step 3 - Creating Arrays for the Features and the Response Variable.

In [ ]:
df.columns

In [ ]:
target_column = ['Outcome']
predictors = list(set(list(df.columns))-set(target_column))
predictors

In [ ]:
# uproszczone skalowanie wartości
df[predictors] = df[predictors]/df[predictors].max()


In [ ]:
df[predictors].head()

In [ ]:
df.describe()

In [ ]:
# tworzy tablice np.array
X = df[predictors].values
y = df[target_column].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# podzial zbioru na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print('X_train:', X_train.shape); print('X_test:', X_test.shape);print('y_train:', y_train.shape);print('y_test:',y_test.shape)

In [ ]:
#Z uwagi na to, że zmienna docelowa reprezentuje kategorię binarną, która została zakodowana jako liczby 0 i 1,
#będziemy musieli ją zakodować.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train[:10]

In [ ]:
count_classes = y_test.shape[1]
print('Liczba klas:', count_classes)

In [ ]:
print(y_train.shape)
y_train

**Definicja, kompilacja i uczenie modelu neuronowego**

In [ ]:
X_train.shape

In [ ]:
#definicja modelu
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=8)) # objaśnić input_dim
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))# objaśnić funkcje aktywacji softmax


# Podczas pracy nad problemem uczenia maszynowego lub głębokiego uczenia,
# funkcje strat/kosztów są wykorzystywane do optymalizacji modelu podczas szkolenia.
# Celem jest prawie zawsze zminimalizowanie funkcji strat.
# Im niższa strata, tym lepszy model. Cross-Entropy loss jest najważniejszą funkcją kosztu.
# Służy ona do optymalizacji modeli klasyfikacyjnych.

In [ ]:
# kompilacja modelu
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# uczenie modelu
model.fit(X_train, y_train, epochs=20, verbose=1)

**Ewaluacja modelu**

In [ ]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))

pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

In [ ]:
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)

In [ ]:
conf_test=confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [ ]:
ax = sns.heatmap(conf_test/np.sum(conf_test), annot=True,
            fmt='.2%', cmap='Blues')

ax.set_title('Test Data');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

plt.show()

In [ ]:
print(classification_report(y_test, y_pred))